In [42]:
import pandas as pd
from sqlalchemy import create_engine, inspect
import sys
from pathlib import Path
from IPython.display import display

In [ ]:
# PostgreSQL Configuration
PG_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "olap",
    "user": "postgres",
    "password": "aa"
}

CSV_PATH = "../data/row_data.csv"

In [25]:
# Create database connection
conn_str = f"postgresql://{PG_CONFIG['user']}:{PG_CONFIG['password']}@{PG_CONFIG['host']}:{PG_CONFIG['port']}/{PG_CONFIG['dbname']}"
engine = create_engine(conn_str)

print("✅ Configuration loaded")
print(f"📁 CSV Path: {CSV_PATH}")
print(f"🗄️  Database: {PG_CONFIG['dbname']} @ {PG_CONFIG['host']}")

✅ Configuration loaded
📁 CSV Path: ../data/row_data.csv
🗄️  Database: olap @ localhost


## 📥 Step 1: Load Raw Data

In [26]:
# Load CSV file
print("📥 Loading row_data.csv...")
df = pd.read_csv(CSV_PATH)

# Clean column names (lowercase, remove special chars)
df.columns = df.columns.str.strip().str.lower().str.replace(r'[^a-z0-9_]', '_', regex=True)

# Parse dates
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)

print(f"✅ Loaded {len(df):,} rows")
print(f"\n📋 Columns: {list(df.columns)}")

# Show sample data
print("\n🔍 Sample Data:")
display(df.head())

📥 Loading row_data.csv...
✅ Loaded 9,800 rows

📋 Columns: ['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'customer_name', 'segment', 'country', 'city', 'state', 'postal_code', 'region', 'product_id', 'category', 'sub_category', 'product_name', 'sales']

🔍 Sample Data:


,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales
0,1,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,2017-06-12,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,2016-10-11,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,2016-10-11,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


## 🏗️ Step 2: Build Dimension Tables

### 📅 DIM_TIME - Date Dimension

In [27]:
print("📅 Building DIM_TIME...")

# Extract unique dates
time_df = df[['order_date']].drop_duplicates().copy()
time_df.rename(columns={'order_date': 'date_full'}, inplace=True)

# Add time attributes
time_df['year'] = time_df['date_full'].dt.year
time_df['quarter'] = time_df['date_full'].dt.quarter
time_df['month'] = time_df['date_full'].dt.month
time_df['week'] = time_df['date_full'].dt.isocalendar().week
time_df['day'] = time_df['date_full'].dt.day
time_df['day_name'] = time_df['date_full'].dt.day_name()
time_df['month_name'] = time_df['date_full'].dt.month_name()
time_df['quarter_name'] = 'Q' + time_df['quarter'].astype(str)

# Load to database
time_df.to_sql('dim_time', engine, if_exists='replace', index=False)

print(f"✅ Created DIM_TIME with {len(time_df)} unique dates")
display(time_df.head())

📅 Building DIM_TIME...
✅ Created DIM_TIME with 1230 unique dates


,date_full,year,quarter,month,week,day,day_name,month_name,quarter_name
0,2017-11-08,2017,4,11,45,8,Wednesday,November,Q4
2,2017-06-12,2017,2,6,24,12,Monday,June,Q2
3,2016-10-11,2016,4,10,41,11,Tuesday,October,Q4
5,2015-06-09,2015,2,6,24,9,Tuesday,June,Q2
12,2018-04-15,2018,2,4,15,15,Sunday,April,Q2


### 👥 DIM_CUSTOMER - Customer Dimension

In [28]:
print("👥 Building DIM_CUSTOMER...")

# Extract unique customers
customer_df = df[['customer_id', 'customer_name', 'segment']].drop_duplicates().copy()

# Load to database
customer_df.to_sql('dim_customer', engine, if_exists='replace', index=False)

print(f"✅ Created DIM_CUSTOMER with {len(customer_df)} unique customers")
print(f"\n📊 Customer Segments:")
print(customer_df['segment'].value_counts())
display(customer_df.head())

👥 Building DIM_CUSTOMER...
✅ Created DIM_CUSTOMER with 793 unique customers

📊 Customer Segments:
segment
Consumer       409
Corporate      236
Home Office    148
Name: count, dtype: int64


,customer_id,customer_name,segment
0,CG-12520,Claire Gute,Consumer
2,DV-13045,Darrin Van Huff,Corporate
3,SO-20335,Sean O'Donnell,Consumer
5,BH-11710,Brosina Hoffman,Consumer
12,AA-10480,Andrew Allen,Consumer


### 🌍 DIM_GEOGRAPHY - Location Dimension

In [29]:
print("🌍 Building DIM_GEOGRAPHY...")

# Extract unique locations
geo_df = df[['region', 'state', 'city', 'country', 'postal_code']].drop_duplicates().copy()

# Load to database
geo_df.to_sql('dim_geography', engine, if_exists='replace', index=False)

print(f"✅ Created DIM_GEOGRAPHY with {len(geo_df)} unique locations")
print(f"\n📊 Regions:")
print(geo_df['region'].value_counts())
display(geo_df.head())

🌍 Building DIM_GEOGRAPHY...
✅ Created DIM_GEOGRAPHY with 628 unique locations

📊 Regions:
region
Central    194
West       180
South      135
East       119
Name: count, dtype: int64


,region,state,city,country,postal_code
0,South,Kentucky,Henderson,United States,42420.0
2,West,California,Los Angeles,United States,90036.0
3,South,Florida,Fort Lauderdale,United States,33311.0
5,West,California,Los Angeles,United States,90032.0
12,South,North Carolina,Concord,United States,28027.0


### 📦 DIM_PRODUCT - Product Dimension

In [30]:
print("📦 Building DIM_PRODUCT...")

# Extract unique products
product_df = df[['product_id', 'category', 'sub_category', 'product_name']].drop_duplicates().copy()

# Load to database
product_df.to_sql('dim_product', engine, if_exists='replace', index=False)

print(f"✅ Created DIM_PRODUCT with {len(product_df)} unique products")
print(f"\n📊 Categories:")
print(product_df['category'].value_counts())
display(product_df.head())

📦 Building DIM_PRODUCT...
✅ Created DIM_PRODUCT with 1893 unique products

📊 Categories:
category
Office Supplies    1097
Technology          413
Furniture           383
Name: count, dtype: int64


,product_id,category,sub_category,product_name
0,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase
1,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,..."
2,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...
3,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table
4,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System


## 🔗 Step 3: Build Fact Table with Foreign Keys

In [34]:
# %%
print("🔗 Building FACT_SALES with foreign key relationships...")

# Read dimension keys from database (using actual column names)
time_keys = pd.read_sql("SELECT date_full FROM dim_time", engine)
time_keys['time_key'] = range(1, len(time_keys) + 1)
time_keys = time_keys.set_index('date_full')['time_key']

customer_keys = pd.read_sql("SELECT customer_id FROM dim_customer", engine)
customer_keys['customer_key'] = range(1, len(customer_keys) + 1)
customer_keys = customer_keys.set_index('customer_id')['customer_key']

# For geography, we need a composite key since region alone isn't unique
geo_dim = pd.read_sql("SELECT region, state, city, country, postal_code FROM dim_geography", engine)
geo_dim['geo_key'] = range(1, len(geo_dim) + 1)
# Create composite key for matching
geo_dim['geo_composite'] = (geo_dim['region'].astype(str) + '|' + 
                             geo_dim['state'].astype(str) + '|' + 
                             geo_dim['city'].astype(str) + '|' + 
                             geo_dim['postal_code'].astype(str))
geo_keys = geo_dim.set_index('geo_composite')['geo_key']

# For products, create composite key with all identifying columns
product_dim = pd.read_sql("SELECT product_id, category, sub_category, product_name FROM dim_product", engine)
product_dim['product_key'] = range(1, len(product_dim) + 1)
# Create composite key for matching
product_dim['product_composite'] = (product_dim['product_id'].astype(str) + '|' + 
                                    product_dim['category'].astype(str) + '|' + 
                                    product_dim['sub_category'].astype(str) + '|' + 
                                    product_dim['product_name'].astype(str))
product_keys = product_dim.set_index('product_composite')['product_key']

# Create fact table
fact_df = df.copy()

# Create composite keys in fact table for matching
fact_df['geo_composite'] = (fact_df['region'].astype(str) + '|' + 
                            fact_df['state'].astype(str) + '|' + 
                            fact_df['city'].astype(str) + '|' + 
                            fact_df['postal_code'].astype(str))

fact_df['product_composite'] = (fact_df['product_id'].astype(str) + '|' + 
                                fact_df['category'].astype(str) + '|' + 
                                fact_df['sub_category'].astype(str) + '|' + 
                                fact_df['product_name'].astype(str))

# Map foreign keys
fact_df['time_key'] = fact_df['order_date'].map(time_keys)
fact_df['customer_key'] = fact_df['customer_id'].map(customer_keys)
fact_df['geo_key'] = fact_df['geo_composite'].map(geo_keys)
fact_df['product_key'] = fact_df['product_composite'].map(product_keys)

# Select fact columns
fact_cols = ['time_key', 'customer_key', 'geo_key', 'product_key', 
             'order_id', 'ship_mode', 'sales']
fact_final = fact_df[fact_cols]

# Load to database
fact_final.to_sql('fact_sales', engine, if_exists='replace', index=False)

print(f"✅ Created FACT_SALES with {len(fact_final):,} rows")
display(fact_final.head())

🔗 Building FACT_SALES with foreign key relationships...
✅ Created FACT_SALES with 9,800 rows


,time_key,customer_key,geo_key,product_key,order_id,ship_mode,sales
0,1,1,1,1,CA-2017-152156,Second Class,261.9600
1,1,1,1,2,CA-2017-152156,Second Class,731.9400
2,2,2,2,3,CA-2017-138688,Second Class,14.6200
3,3,3,3,4,US-2016-108966,Standard Class,957.5775
4,3,3,3,5,US-2016-108966,Standard Class,22.3680


## ✅ Step 4: Verify Star Schema

In [35]:
print("🔍 Verifying Star Schema...")

# Count records in each table
verification = {
    'fact_sales': pd.read_sql("SELECT COUNT(*) as count FROM fact_sales", engine).iloc[0, 0],
    'dim_time': pd.read_sql("SELECT COUNT(*) as count FROM dim_time", engine).iloc[0, 0],
    'dim_customer': pd.read_sql("SELECT COUNT(*) as count FROM dim_customer", engine).iloc[0, 0],
    'dim_geography': pd.read_sql("SELECT COUNT(*) as count FROM dim_geography", engine).iloc[0, 0],
    'dim_product': pd.read_sql("SELECT COUNT(*) as count FROM dim_product", engine).iloc[0, 0],
}

# Display results
verification_df = pd.DataFrame(list(verification.items()), columns=['Table', 'Row Count'])
verification_df['Row Count'] = verification_df['Row Count'].apply(lambda x: f"{x:,}")

print("\n📊 Star Schema Summary:")
display(verification_df)

print("\n🎉 ETL Pipeline Complete!")
print("✅ All dimension and fact tables loaded successfully")


🔍 Verifying Star Schema...

📊 Star Schema Summary:


,Table,Row Count
0,fact_sales,"9,800"
1,dim_time,"1,230"
2,dim_customer,793
3,dim_geography,628
4,dim_product,"1,893"



🎉 ETL Pipeline Complete!
✅ All dimension and fact tables loaded successfully


## 🔬 Bonus: Sample Query

In [37]:
# %% [markdown]
## 🔬 Bonus: Sample Query

# %%
# Example: Total sales by category
# Note: Since dimension tables don't have keys stored, we'll query fact table directly
query = """
SELECT 
    COUNT(DISTINCT f.order_id) as orders,
    ROUND(SUM(f.sales)::numeric, 2) as total_sales
FROM fact_sales f
"""

result = pd.read_sql(query, engine)
print("📊 Total Sales Summary:")
display(result)

# Better approach: Join fact with original data through composite matching
print("\n📊 Sales by Category (using Python aggregation):")

# Read fact table
fact = pd.read_sql("SELECT * FROM fact_sales", engine)

# Read product dimension
products = pd.read_sql("SELECT * FROM dim_product", engine)
products['product_key'] = range(1, len(products) + 1)

# Merge and aggregate
merged = fact.merge(products, on='product_key', how='left')
category_sales = merged.groupby('category').agg({
    'order_id': 'nunique',
    'sales': 'sum'
}).reset_index()
category_sales.columns = ['Category', 'Orders', 'Total Sales']
category_sales['Total Sales'] = category_sales['Total Sales'].round(2)
category_sales = category_sales.sort_values('Total Sales', ascending=False)

display(category_sales)

📊 Total Sales Summary:


,orders,total_sales
0,4922,2261536.78



📊 Sales by Category (using Python aggregation):


,Category,Orders,Total Sales
2,Technology,1519,827455.87
0,Furniture,1727,728658.58
1,Office Supplies,3676,705422.33


In [38]:
print("=" * 80)
print("🌟 STAR SCHEMA VERIFICATION REPORT")
print("=" * 80)

🌟 STAR SCHEMA VERIFICATION REPORT


In [45]:
inspector = inspect(engine)
tables = inspector.get_table_names()
print(f"\n📊 Tables Found: {len(tables)}")
for i, table in enumerate(tables, 1):
    print(f"  {i}. {table}")


📊 Tables Found: 5
  1. dim_time
  2. dim_customer
  3. dim_geography
  4. dim_product
  5. fact_sales


In [46]:
# Expected tables
expected_tables = ['fact_sales', 'dim_time', 'dim_customer', 'dim_geography', 'dim_product']
missing_tables = set(expected_tables) - set(tables)

if missing_tables:
    print(f"\n⚠️  Missing tables: {missing_tables}")
else:
    print(f"\n✅ All expected tables exist!")



✅ All expected tables exist!


In [47]:
print("\n" + "=" * 80)
print("📐 SCHEMA STRUCTURE")
print("=" * 80)


📐 SCHEMA STRUCTURE


In [48]:
for table in expected_tables:
    if table not in tables:
        print(f"\n❌ {table.upper()} - NOT FOUND")
        continue
    
    print(f"\n{'='*60}")
    print(f"📋 {table.upper()}")
    print(f"{'='*60}")
    
    # Get columns
    columns = inspector.get_columns(table)
    
    # Display in table format
    col_data = []
    for col in columns:
        col_data.append({
            'Column': col['name'],
            'Type': str(col['type']),
            'Nullable': 'Yes' if col['nullable'] else 'No'
        })
    
    col_df = pd.DataFrame(col_data)
    display(col_df)
    
    # Get row count
    count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table}", engine).iloc[0, 0]
    print(f"\n📊 Total Rows: {count:,}")



📋 FACT_SALES


,Column,Type,Nullable
0,time_key,BIGINT,Yes
1,customer_key,BIGINT,Yes
2,geo_key,BIGINT,Yes
3,product_key,BIGINT,Yes
4,order_id,TEXT,Yes
5,ship_mode,TEXT,Yes
6,sales,DOUBLE PRECISION,Yes



📊 Total Rows: 9,800

📋 DIM_TIME


,Column,Type,Nullable
0,date_full,TIMESTAMP,Yes
1,year,INTEGER,Yes
2,quarter,INTEGER,Yes
3,month,INTEGER,Yes
4,week,BIGINT,Yes
5,day,INTEGER,Yes
6,day_name,TEXT,Yes
7,month_name,TEXT,Yes
8,quarter_name,TEXT,Yes



📊 Total Rows: 1,230

📋 DIM_CUSTOMER


...


📊 Total Rows: 793

📋 DIM_GEOGRAPHY


,Column,Type,Nullable
0,region,TEXT,Yes
1,state,TEXT,Yes
2,city,TEXT,Yes
3,country,TEXT,Yes
4,postal_code,DOUBLE PRECISION,Yes



📊 Total Rows: 628

📋 DIM_PRODUCT


,Column,Type,Nullable
0,product_id,TEXT,Yes
1,category,TEXT,Yes
2,sub_category,TEXT,Yes
3,product_name,TEXT,Yes



📊 Total Rows: 1,893


In [49]:
### 📊 3. Row Count Summary

print("\n" + "=" * 80)
print("📊 ROW COUNT SUMMARY")
print("=" * 80)

row_counts = []
for table in expected_tables:
    if table in tables:
        count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table}", engine).iloc[0, 0]
        row_counts.append({'Table': table, 'Row Count': f"{count:,}"})
    else:
        row_counts.append({'Table': table, 'Row Count': 'NOT FOUND'})

summary_df = pd.DataFrame(row_counts)
display(summary_df)



📊 ROW COUNT SUMMARY


,Table,Row Count
0,fact_sales,"9,800"
1,dim_time,"1,230"
2,dim_customer,793
3,dim_geography,628
4,dim_product,"1,893"


In [50]:
### 🔗 4. Foreign Key Integrity Check

print("\n" + "=" * 80)
print("🔗 FOREIGN KEY INTEGRITY CHECK")
print("=" * 80)

if 'fact_sales' in tables:
    fact = pd.read_sql("SELECT * FROM fact_sales LIMIT 1000", engine)
    
    print(f"\n✅ Checking FACT_SALES (sample of {len(fact):,} rows)")
    print("-" * 60)
    
    # Check for NULL foreign keys
    fk_columns = ['time_key', 'customer_key', 'geo_key', 'product_key']
    
    integrity_results = []
    
    for fk in fk_columns:
        if fk in fact.columns:
            null_count = fact[fk].isna().sum()
            total = len(fact)
            null_pct = (null_count / total * 100) if total > 0 else 0
            
            status = "✅ OK" if null_count == 0 else f"⚠️  {null_count} NULLs"
            
            integrity_results.append({
                'Foreign Key': fk,
                'NULL Count': null_count,
                'NULL %': f"{null_pct:.2f}%",
                'Status': status
            })
    
    integrity_df = pd.DataFrame(integrity_results)
    display(integrity_df)
    
    # Check for negative or zero keys
    print("\n🔍 Checking for invalid key values (≤ 0):")
    for fk in fk_columns:
        if fk in fact.columns:
            invalid = (fact[fk] <= 0).sum()
            if invalid > 0:
                print(f"  ⚠️  {fk}: {invalid} invalid values")
            else:
                print(f"  ✅ {fk}: All values valid")



🔗 FOREIGN KEY INTEGRITY CHECK

✅ Checking FACT_SALES (sample of 1,000 rows)
------------------------------------------------------------


,Foreign Key,NULL Count,NULL %,Status
0,time_key,0,0.00%,✅ OK
1,customer_key,0,0.00%,✅ OK
2,geo_key,0,0.00%,✅ OK
3,product_key,0,0.00%,✅ OK



🔍 Checking for invalid key values (≤ 0):
  ✅ time_key: All values valid
  ✅ customer_key: All values valid
  ✅ geo_key: All values valid
  ✅ product_key: All values valid


In [51]:
### 📈 5. Data Quality Checks

print("\n" + "=" * 80)
print("📈 DATA QUALITY CHECKS")
print("=" * 80)

quality_checks = []

# Check 1: Fact table has data
if 'fact_sales' in tables:
    fact_count = pd.read_sql("SELECT COUNT(*) as count FROM fact_sales", engine).iloc[0, 0]
    quality_checks.append({
        'Check': 'Fact table populated',
        'Result': '✅ PASS' if fact_count > 0 else '❌ FAIL',
        'Details': f"{fact_count:,} rows"
    })

# Check 2: All dimensions have data
for dim_table in ['dim_time', 'dim_customer', 'dim_geography', 'dim_product']:
    if dim_table in tables:
        dim_count = pd.read_sql(f"SELECT COUNT(*) as count FROM {dim_table}", engine).iloc[0, 0]
        quality_checks.append({
            'Check': f'{dim_table} populated',
            'Result': '✅ PASS' if dim_count > 0 else '❌ FAIL',
            'Details': f"{dim_count:,} rows"
        })

# Check 3: No duplicate dates in dim_time
if 'dim_time' in tables:
    dup_query = "SELECT date_full, COUNT(*) as cnt FROM dim_time GROUP BY date_full HAVING COUNT(*) > 1"
    duplicates = pd.read_sql(dup_query, engine)
    quality_checks.append({
        'Check': 'dim_time has unique dates',
        'Result': '✅ PASS' if len(duplicates) == 0 else '⚠️  WARNING',
        'Details': f"{len(duplicates)} duplicates found" if len(duplicates) > 0 else "No duplicates"
    })

# Check 4: Sales values are reasonable
if 'fact_sales' in tables:
    sales_stats = pd.read_sql("SELECT MIN(sales) as min_sales, MAX(sales) as max_sales, AVG(sales) as avg_sales FROM fact_sales", engine)
    min_sales = sales_stats.iloc[0, 0]
    max_sales = sales_stats.iloc[0, 1]
    avg_sales = sales_stats.iloc[0, 2]
    
    quality_checks.append({
        'Check': 'Sales values are positive',
        'Result': '✅ PASS' if min_sales > 0 else '⚠️  WARNING',
        'Details': f"Min: ${min_sales:.2f}, Max: ${max_sales:.2f}, Avg: ${avg_sales:.2f}"
    })

quality_df = pd.DataFrame(quality_checks)
display(quality_df)


📈 DATA QUALITY CHECKS


,Check,Result,Details
0,Fact table populated,✅ PASS,"9,800 rows"
1,dim_time populated,✅ PASS,"1,230 rows"
2,dim_customer populated,✅ PASS,793 rows
3,dim_geography populated,✅ PASS,628 rows
4,dim_product populated,✅ PASS,"1,893 rows"
5,dim_time has unique dates,✅ PASS,No duplicates
6,Sales values are positive,✅ PASS,"Min: $0.44, Max: $22638.48, Avg: $230.77"


In [52]:
### 🎨 6. Star Schema Visualization (Text-based)

# %%
print("\n" + "=" * 80)
print("🌟 STAR SCHEMA DIAGRAM")
print("=" * 80)

schema_diagram = """
                        ┌─────────────────┐
                        │   DIM_TIME      │
                        ├─────────────────┤
                        │ • date_full     │
                        │ • year          │
                        │ • quarter       │
                        │ • month         │
                        │ • week          │
                        │ • day           │
                        └────────┬────────┘
                                 │
                                 │ time_key
                                 │
    ┌─────────────────┐          │          ┌─────────────────┐
    │ DIM_CUSTOMER    │          │          │ DIM_PRODUCT     │
    ├─────────────────┤          │          ├─────────────────┤
    │ • customer_id   │          │          │ • product_id    │
    │ • customer_name │          │          │ • category      │
    │ • segment       │          │          │ • sub_category  │
    └────────┬────────┘          │          │ • product_name  │
             │                   │          └────────┬────────┘
             │                   │                   │
             │ customer_key      │                   │ product_key
             │                   │                   │
             │            ┌──────▼──────┐            │
             └───────────►│ FACT_SALES  │◄───────────┘
                          ├─────────────┤
                          │ • time_key  │
                          │ • customer_key
                          │ • geo_key   │
                          │ • product_key
                          │ • order_id  │
                          │ • ship_mode │
                          │ • sales     │
                          └──────┬──────┘
                                 │
                                 │ geo_key
                                 │
                        ┌────────▼────────┐
                        │ DIM_GEOGRAPHY   │
                        ├─────────────────┤
                        │ • region        │
                        │ • state         │
                        │ • city          │
                        │ • country       │
                        │ • postal_code   │
                        └─────────────────┘
"""

print(schema_diagram)


🌟 STAR SCHEMA DIAGRAM

                        ┌─────────────────┐
                        │   DIM_TIME      │
                        ├─────────────────┤
                        │ • date_full     │
                        │ • year          │
                        │ • quarter       │
                        │ • month         │
                        │ • week          │
                        │ • day           │
                        └────────┬────────┘
                                 │
                                 │ time_key
                                 │
    ┌─────────────────┐          │          ┌─────────────────┐
    │ DIM_CUSTOMER    │          │          │ DIM_PRODUCT     │
    ├─────────────────┤          │          ├─────────────────┤
    │ • customer_id   │          │          │ • product_id    │
    │ • customer_name │          │          │ • category      │
    │ • segment       │          │          │ • sub_category  │
    └────────┬────────┘          │    

In [53]:
### ✅ 7. Final Validation Summary

print("\n" + "=" * 80)
print("✅ FINAL VALIDATION SUMMARY")
print("=" * 80)

validation_summary = {
    'Total Tables': len(tables),
    'Expected Tables': len(expected_tables),
    'Tables Match': '✅ Yes' if set(expected_tables).issubset(set(tables)) else '❌ No',
}

if 'fact_sales' in tables:
    fact_count = pd.read_sql("SELECT COUNT(*) as count FROM fact_sales", engine).iloc[0, 0]
    validation_summary['Fact Records'] = f"{fact_count:,}"

if all(dim in tables for dim in ['dim_time', 'dim_customer', 'dim_geography', 'dim_product']):
    dim_counts = []
    for dim in ['dim_time', 'dim_customer', 'dim_geography', 'dim_product']:
        count = pd.read_sql(f"SELECT COUNT(*) as count FROM {dim}", engine).iloc[0, 0]
        dim_counts.append(count)
    validation_summary['Dimension Records'] = f"{sum(dim_counts):,}"

# Display summary
print()
for key, value in validation_summary.items():
    print(f"  {key:.<40} {value}")

print("\n" + "=" * 80)

# Count issues
issues = 0
if missing_tables:
    issues += len(missing_tables)

if 'fact_sales' in tables:
    fact_sample = pd.read_sql("SELECT * FROM fact_sales LIMIT 100", engine)
    for fk in ['time_key', 'customer_key', 'geo_key', 'product_key']:
        if fk in fact_sample.columns:
            if fact_sample[fk].isna().sum() > 0:
                issues += 1

if issues == 0:
    print("🎉 PERFECT! Star schema is complete and valid!")
    print("✅ All tables exist, all foreign keys are populated")
else:
    print(f"⚠️  Found {issues} issue(s) that need attention")
    print("📝 Review the checks above for details")

print("=" * 80)


✅ FINAL VALIDATION SUMMARY

  Total Tables............................ 5
  Expected Tables......................... 5
  Tables Match............................ ✅ Yes
  Fact Records............................ 9,800
  Dimension Records....................... 4,544

🎉 PERFECT! Star schema is complete and valid!
✅ All tables exist, all foreign keys are populated
